# Problem
Can the sale amount for properties in Conneticut be predicted?

Also, where are the best places to buy? Best places to sell? places with the best/worst sale ratios?

Through understanding this dataset, real estate owners and sellers in Conneticut can make smart sales and purchases.

# Data Collection
Data sourced from data.gov

https://catalog.data.gov/dataset/real-estate-sales-2001-2018

In [1]:
import pandas as pd
df = pd.read_csv('Real_Estate_Sales_2001-2023_GL.csv')

C:\Users\jonah\AppData\Local\Temp\ipykernel_12092\216775831.py:2: DtypeWarning: Columns (7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Real_Estate_Sales_2001-2023_GL.csv')


# Data Understanding

In [2]:
df.head()

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,04/14/2021,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,05/26/2021,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN
2,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.463,Commercial,NaN,NaN,NaN,NaN,NaN
3,2020090,2020,12/14/2020,Ansonia,57 PLATT ST,127400.0,202500.0,0.6291,Residential,Two Family,NaN,NaN,NaN,NaN
4,210288,2021,06/20/2022,Avon,12 BYRON DRIVE,179990.0,362500.0,0.4965,Residential,Condo,NaN,NaN,NaN,POINT (-72.879115982 41.773452988)


In [3]:
df.tail()

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
1141717,230299,2023,04/19/2024,Southington,580-23 MERIDEN WATERBURY TPKE,26540.0,509900.0,0.0520,Residential,Condo,07 - Change in Property,NEW CONSTRUCTION OVER 55 CONDO,NaN,POINT (-72.86373 41.56032)
1141718,230568,2023,05/15/2024,West Haven,165 MILTON AVE,148050.0,400300.0,0.3698,Residential,Single Family,NaN,NaN,NaN,POINT (-72.97401 41.24665)
1141719,230217,2023,02/20/2024,Southington,242 LAZY LN,177340.0,334750.0,0.5297,Residential,Single Family,NaN,NaN,NaN,POINT (-72.88193 41.62094)
1141720,230097,2023,05/30/2024,Old Lyme,21 BUTTONBALL RD,8030.0,35000.0,0.2294,Vacant Land,NaN,NaN,DEVELOPERS LOT FOR NEW CONST,NaN,POINT (-72.30557 41.29054)
1141721,230383,2023,06/06/2024,Shelton,512 ANTELOPE TRL,306460.0,659813.0,0.4645,Residential,Condo,25 - Other,89 33 512,"PER MLS CLOSING PRICE = $675,000",POINT (-73.13448 41.29345)


In [4]:
df.shape

(1141722, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1141722 entries, 0 to 1141721
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Serial Number     1141722 non-null  int64  
 1   List Year         1141722 non-null  int64  
 2   Date Recorded     1141720 non-null  object 
 3   Town              1141722 non-null  object 
 4   Address           1141671 non-null  object 
 5   Assessed Value    1141722 non-null  float64
 6   Sale Amount       1141722 non-null  float64
 7   Sales Ratio       1141722 non-null  object 
 8   Property Type     759276 non-null   object 
 9   Residential Type  738804 non-null   object 
 10  Non Use Code      324807 non-null   object 
 11  Assessor Remarks  181090 non-null   object 
 12  OPM remarks       14346 non-null    object 
 13  Location          341241 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 121.9+ MB


In [30]:
print('Null values:')
for col in df.columns:
    print(f'{col}: {df.shape[0]-df[col].count()}')

Null values:
Serial Number: 0
List Year: 0
Date Recorded: 2
Town: 0
Address: 51
Assessed Value: 0
Sale Amount: 0
Sales Ratio: 0
Property Type: 382446
Residential Type: 402918
Non Use Code: 816915
Assessor Remarks: 960632
OPM remarks: 1127376
Location: 800481


Values like Non Use Code, Assessor Remarks, OPM remarks, Location have over half of their entrances missing

In [32]:
print('Null values:')
for col in df.columns:
    print(f'{df[col].apply(type).value_counts()}')

Null values:
Serial Number
<class 'int'>    1141722
Name: count, dtype: int64
List Year
<class 'int'>    1141722
Name: count, dtype: int64
Date Recorded
<class 'str'>      1141720
<class 'float'>          2
Name: count, dtype: int64
Town
<class 'str'>    1141722
Name: count, dtype: int64
Address
<class 'str'>      1141671
<class 'float'>         51
Name: count, dtype: int64
Assessed Value
<class 'float'>    1141722
Name: count, dtype: int64
Sale Amount
<class 'float'>    1141722
Name: count, dtype: int64
Sales Ratio
<class 'float'>    1141722
Name: count, dtype: int64
Property Type
<class 'str'>      759276
<class 'float'>    382446
Name: count, dtype: int64
Residential Type
<class 'str'>      738804
<class 'float'>    402918
Name: count, dtype: int64
Non Use Code
<class 'float'>    862445
<class 'str'>      279277
Name: count, dtype: int64
Assessor Remarks
<class 'float'>    960632
<class 'str'>      181090
Name: count, dtype: int64
OPM remarks
<class 'float'>    1127376
<class 'str'>

# Preprocessing

## Data Cleaning

Sales Ratio should contain only floats, but df.info() shows it to have an object Dtype. Why is this?

In [6]:
df['Sales Ratio'].apply(type).value_counts()

Sales Ratio
<class 'float'>    1114112
<class 'str'>        27610
Name: count, dtype: int64

Since this column contains floats and strings, we need to convert all of the strings to floats

In [7]:
def is_float(x):
    try:
        float(x)
        return True
    except:
        print(x)
        return False
sales_ratio_strings = df['Sales Ratio'][df['Sales Ratio'].apply(lambda x: not isinstance(x, float))]
sales_ratio_strings.apply(is_float).value_counts()

1,787.3333


Sales Ratio
True     27609
False        1
Name: count, dtype: int64

Of all the strings in df['Sales Ratio'], there is only one which cannot be converted to a float, so we can fix this individual datapoint.

Now every element in df['Sales Ratio'] is a float

In [8]:
df['Sales Ratio'] = df['Sales Ratio'].replace('1,787.3333','1787.3333')
df['Sales Ratio'] = pd.to_numeric(df['Sales Ratio'])
df['Sales Ratio'].apply(type).value_counts()

Sales Ratio
<class 'float'>    1141722
Name: count, dtype: int64

# Model Planning

# Model Building

# Results